In [1]:

import os,sys #user to create and modify file name and save it
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp #ReadingShapefile
import glob                                                                 
import xlsxwriter #for writing in Excel
warnings.filterwarnings('ignore')

#### Data Reading

In [2]:
os.chdir('F:/Dataset_New/Nd')
ds=xr.open_dataset('Nd_PreMon.nc')
Nd=ds.__xarray_dataarray_variable__

lon, lat, time=Nd.indexes.values()
lat=lat.values
lon=lon.values


### Visualizing the DataPoints

In [5]:
import matplotlib.pyplot as plt
from cartopy import config
import cartopy.crs as ccrs

#Makind A Coodinate array
coord=[]
for x in lon:
    for y in lat:
        coord.append([x,y])
coord=np.array(coord)

#Reading the Shape File
ax = plt.axes(projection=ccrs.PlateCarree())
shapefile ="E:\Arun-ACIML\SHPs\BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')

# Ploting
countries.plot(ax=ax,column='featurecla')
ax.scatter(coord[:,0],coord[:,1],color='red')

#Adjsutments

min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

ax.coastlines()
ax.set_xlim([min_lon, max_lon])
ax.set_ylim([min_lat, max_lat])
gridlines = ax.gridlines(draw_labels=True,linewidth=0)
gridlines.top_labels = False
gridlines.right_labels = False

plt.show()

# Sorting out datapoints within the AOI

In [4]:
shapefile ="E:\Arun-ACIML\SHPs\BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')

#Clipping to shape File and Creating Mask File
shapefile ="E:\Arun-ACIML\SHPs\BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

mask=countries_mask_poly.mask(ds,lat_name='latitude',lon_name='longitude')

In [5]:
Nlat=mask.latitude.where(mask==0).values
Nlon=mask.longitude.where(mask==0).values.transpose()

Llat=Nlat.flatten()
Llon=Nlon.flatten()

#Collecting the the coordinates
Nlat=Llat[~np.isnan(Llat)]
Nlon=Llon[~np.isnan(Llon)]

### Verifying the datapoints with Reverse engineering

In [6]:
# set of latitude and longitude pairs
lat_lon_pairs = []
for x in range(0,183):
    lat_lon_pairs.append((Nlat[x], Nlon[x]))
data_values = []

# loop over the latitude and longitude pairs
for lat, lon in lat_lon_pairs:
    # get the indices of the latitude and longitude
    lat_index = np.where(mask.latitude == lat)[0][0]
    lon_index = np.where(mask.longitude == lon)[0][0]
    
    # extract the data value using indexing
    data_value = mask.values[lat_index, lon_index]
    
    # append the data value to the list
    data_values.append(data_value)

data_values